In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_colwidth', None)
import os
home_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
from tqdm import tqdm



index_selected_list = ['上证50','上证150','沪深300','中证500','中证1000','科创50','创业板50','创业300']
ignored_list = ['上证180','上证380','深证100','深证200','深证700']
concepts_list = pd.read_csv(home_path + '/static/concepts_checklist.csv',encoding="utf-8")
industry_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['行业','产品','业务','行业+']) & concepts_list['active']==1 ]['概念名称'].values.tolist()
events_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['事件','改革','政策']) & concepts_list['active']==1]['概念名称'].values.tolist()
master_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['知名企业','国企体系']) & concepts_list['active']==1]['概念名称'].values.tolist()
exclude_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['交易'])]['概念名称'].values.tolist()

wc_result = pd.DataFrame()

for i in tqdm(range(1,2)):
    print('page: '+ str(i))
    sr = wc.search('2023年预测净利润 总股本 行业 所属指数类 公司亮点',page =i).fillna('')
    sr['沪深指数'] = ''
    sr['行业概念'] = ''
    sr['事件概念'] = ''
    sr['体系概念'] = ''

    for row, l in sr.iterrows():
        # sr['主营产品名称'][row] = ','.join(l['主营产品名称'].split('||'))
        sr['沪深指数'][row] = ','.join(set(l['所属指数类'].split(';')).intersection(index_selected_list))
        sr['所属概念'][row] = ','.join(set(l['所属概念'].split(';')).difference(exclude_concepts_list))
        # sr['行业概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(industry_concepts_list)).replace('金属','')
        # sr['事件概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(events_concepts_list))
        # sr['体系概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(master_concepts_list))

    sr['所属概念'] =   sr['所属概念'].str.replace('概念股','').str.replace('概念','')
#     sr['流动市值'] = (pd.to_numeric(sr['a股市值(不含限售股)'])/100000000).round(1).fillna('')    
    

In [2]:
import requests

url = 'http://www.iwencai.com/gateway/urp/v7/landing/getDataList'

query_string = '所属概念 公司亮点 所属同花顺行业 所属指数类'
page = 1

condition = []
condition_1 = {
        'score': 0.0,
        'chunkedResult': '_&_'.join(query_string.split(' ')),
        'opName': 'and',
        'opProperty': '',
        'sonSize': 3,
        'relatedSize': '0',
        'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
        'source': 'text2sql'
        }
condition.append(condition_1)

for i in query_string.split(' '):
#     i = i.replace('指数','指数类')
    dic = {
        'dateText': '',
        'indexName': i,
        'indexProperties': [],
        'child_node': i,
        'ci': True,
        'source': 'text2sql',
        'type': 'index',
        'indexPropertiesMap': {},
        'reportType': 'null',
        'ciChunk': i,
        'createBy': 'preCache',
        'uiText': i,
        'valueType': '_' + i,
        'domain': 'abs_股票领域',
        'sonSize': 0,
        'dateList': []
        }
    condition.append(dic)
condition = str(condition).replace("'", '"').replace(' ','')


payload = {
    "question": query_string,
    "page": page,
    "perpage": 100,
    "source": "Ths_iwencai_Xuangu",
    'urp_sort_way': 'desc',
    'urp_sort_index': '最新涨跌幅',
    'addheaderindexes': '',
    'codelist': '',
    'indexnamelimit': '',
    'ret': 'json_all',
    'urp_use_sort': '1',
    'user_id': '570662655',
    'uuids[0]': '24087',
    'query_type': 'stock',
    'comp_id': '6734520',
    'business_cat': 'soniu',
    'uuid': '24087',
#    以下为每次变化
#     'date_range[0]': '20230610',
#     'iwc_token': '0ac9529816863435305771116',
#     'sessionid': '9f98f2743c41f01ae1e20f42ce38d2a1',
#     'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
    'condition': str(condition).replace("'", '"').replace(' ','')
}

cookies = {
    "WafStatus":'0',
    "ta_random_userid":'anro5tt7uu',
    "cid":'e77a8079986b06e6a9a46e614cf7eee21659209895',
    "ComputerID":'e77a8079986b06e6a9a46e614cf7eee21659209895',
    "other_uid":'Ths_iwencai_Xuangu_x25492z942nd2ag2aqbfqwh7m852m7vi',
    "ttype":'WEB',
    "wencai_pc_version":'1',
    "user_status":'0',
    "user":'MDpteF81NzA2NjI2NTU6Ok5vbmU6NTAwOjU4MDY2MjY1NTo3LDExMTExMTExMTExLDQwOzQ0LDExLDQwOzYsMSw0MDs1LDEsNDA6MTY6Ojo1NzA2NjI2NTU6MTY4NjMyMTg1NTo6OjE2MTQ4OTgzMjA6NjA0ODAwOjA6MTk0Yzg1NGMxMzdlYzY1ZmY1NTcxNjIzMWU2YzA2MGZkOmRlZmF1bHRfNDow',
    "userid":'570662655',
    "u_name":'mx_570662655',
    "escapename":'mx_570662655',
    "ticket":'106f915b0a9985030a58c2151f69d9b8',
    "utk":'86be216097cbd990b074d0a691f39d39',
#     'THSSESSID':'210fbd845858470fd3dcc8c227',
#     "PHPSESSID":'i1jarg6eiikd16j6ajcjuqhb7j1a88je',
#     "v":'A1hilGtTiE5eR6Q8zbqmwNg1KY3vQa3eHoyRWZLcpY9GCvSzOlGMW261YLTh',
}


headers = {
    'Accept':'application/json, text/plain, */*',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,en-US;q=0.7,ja;q=0.6,zh-TW;q=0.5,nl;q=0.4',
    'Cache-Control':'no-cache',
    'Connection':'keep-alive',
    'Content-Length':'2221',
    'Content-Type':'application/x-www-form-urlencoded',
#     'Hexin-V':'A1hilGtTiE5eR6Q8zbqmwNg1KY3vQa3eHoyRWZLcpY9GCvSzOlGMW261YLTh',
    'Host':'www.iwencai.com',
    'Origin':'http://www.iwencai.com',
    'Pragma':'no-cache',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}



r = requests.post(url, data = payload, cookies = cookies, headers = headers)

import json
import pandas as pd
df = pd.DataFrame(r.json()['answer']['components'][0]['data']['datas'])
df

,股票代码,股票简称,最新价,最新涨跌幅,所属概念,公司亮点,所属同花顺行业,所属指数类,market_code,code
0,301315.SZ,威士顿,57.70,78.693,转融券标的;融资融券;人工智能;ERP概念;新股与次新股;注册制次新股;机器人概念;工业4....,在烟草行业的信息化领域具有一定知名度，兼营商业银行数据管理信息化,计算机-计算机应用-软件开发,创业板综;同花顺全A;深证A指;深证综指,33,301315
1,833455.BJ,汇隆活塞,4.77,51.429,新股与次新股,国家级专精特新“小巨人”，主营活塞、缸套等中低速内燃机零部件,机械设备-通用设备-其他通用设备,None,151,833455
2,871981.BJ,晶赛科技,17.13,29.970,专精特新;融资融券,公司处于石英晶振行业第一梯队,电子-半导体及元件-被动元件,中证A股;中证流通;北证50,151,871981
3,430418.BJ,苏轴股份,16.27,29.848,专精特新;地方国企改革;国企改革;融资融券;江苏国企改革;高送转预期,滚针轴承细分领域龙头,交运设备-汽车零部件-汽车零部件Ⅲ,中证A股;中证流通;北证50,151,430418
4,300356.SZ,光一退,1.06,20.455,None,智能用电信息采集系统软硬件的研发、生产和销售领先企业,电力设备-电力设备-电气自控设备,None,37,300356
...,...,...,...,...,...,...,...,...,...,...
95,000837.SZ,秦川机床,15.10,5.965,工业4.0;3D打印;军工;机器人概念;高端装备;工业互联网;工业机器人;一带一路;航空发动...,中国机床工具龙头，国内规格最全、系列最多的工业机器人关节减速器生产制造商,机械设备-通用设备-机床工具,中小创新;中证A股;中证全指;中证流通;制造指数;同花顺全A;新指数;深证1000;深证70...,33,000837
96,603666.SH,亿嘉和,46.96,5.837,无人机;沪股通;标普道琼斯A股;机器人概念;医疗机器人;专精特新;充电桩,机器人领域的高新技术企业,机械设备-自动化设备-机器人,A股指数;上证指数;上证流通;中型综指;中证1000;中证1000USD;中证A股;中证全指...,17,603666
97,838171.BJ,邦德股份,8.93,5.806,汽车热管理;人民币贬值受益;融资融券,None,交运设备-汽车零部件-汽车零部件Ⅲ,中证A股;中证流通;北证50,151,838171
98,603489.SH,八方股份,65.66,5.767,共享单车;沪股通;专精特新;传感器;融资融券;转融券标的;养老金持股,电踏车用电机龙头厂商,电力设备-电力设备-电机,A股指数;上证100;上证380;上证指数;上证流通;中型综指;中证1000;中证1000U...,17,603489


In [165]:
a = '[{"score":0.0,"chunkedResult":"所属概念_&_所属指数_&_公司亮点","opName":"and","opProperty":"","sonSize":3,"relatedSize":"0","logid":"9f98f2743c41f01ae1e20f42ce38d2a1","source":"text2sql"},{"dateText":"","indexName":"所属概念","indexProperties":[],"child_node":"所属概念","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属概念","createBy":"preCache","uiText":"所属概念","valueType":"_所属概念","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"所属指数类","indexProperties":[],"child_node":"所属指数类","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属指数","createBy":"preCache","uiText":"所属指数类","valueType":"_所属指数类","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"公司亮点","indexProperties":[],"child_node":"公司亮点","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"公司亮点","createBy":"preCache","uiText":"公司亮点","valueType":"_公司亮点","domain":"abs_股票领域","sonSize":0,"dateList":[]}]'


In [131]:
query_string = '所属概念 所属指数 公司亮点 所属行业'
'_&_'.join(query_string.split(' '))

'所属概念_&_所属指数_&_公司亮点_&_所属行业'

In [167]:
a

'[{"score":0.0,"chunkedResult":"所属概念_&_所属指数_&_公司亮点","opName":"and","opProperty":"","sonSize":3,"relatedSize":"0","logid":"9f98f2743c41f01ae1e20f42ce38d2a1","source":"text2sql"},{"dateText":"","indexName":"所属概念","indexProperties":[],"child_node":"所属概念","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属概念","createBy":"preCache","uiText":"所属概念","valueType":"_所属概念","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"所属指数类","indexProperties":[],"child_node":"所属指数类","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属指数","createBy":"preCache","uiText":"所属指数类","valueType":"_所属指数类","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"公司亮点","indexProperties":[],"child_node":"公司亮点","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"公司亮点","createBy":"preCache","uiText":"公司亮点","valueType":"_公司亮点","domain"

In [170]:
str(condition).replace("'", '"').replace(' ','')

'[{"score":0.0,"chunkedResult":"所属概念_&_所属指数_&_公司亮点","opName":"and","opProperty":"","sonSize":3,"relatedSize":"0","logid":"9f98f2743c41f01ae1e20f42ce38d2a1","source":"text2sql"},{"dateText":"","indexName":"所属概念","indexProperties":[],"child_node":"所属概念","ci":True,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属概念","createBy":"preCache","uiText":"所属概念","valueType":"_所属概念","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"所属指数","indexProperties":[],"child_node":"所属指数","ci":True,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属指数","createBy":"preCache","uiText":"所属指数","valueType":"_所属指数类","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"公司亮点","indexProperties":[],"child_node":"公司亮点","ci":True,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"公司亮点","createBy":"preCache","uiText":"公司亮点","valueType":"_公司亮点","domain":"a

In [ ]:



{'score': 0.0,
  'chunkedResult': '_&_'.join(query_string.split(' ')),
  'opName': 'and',
  'opProperty': '',
  'sonSize': 3,
  'relatedSize': '0',
#   'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
  'source': 'text2sql'}


{'dateText': '',
  'indexName': '所属概念',
  'indexProperties': [],
  'child_node': '所属概念',
  'ci': True,
  'source': 'text2sql',
  'type': 'index',
  'indexPropertiesMap': {},
  'reportType': 'null',
  'ciChunk': '所属概念',
  'createBy': 'preCache',
  'uiText': '所属概念',
  'valueType': '_所属概念',
  'domain': 'abs_股票领域',
  'sonSize': 0,
  'dateList': []},

In [128]:
json.loads(a[0])

[{'score': 0.0,
  'chunkedResult': '所属概念_&_所属指数_&_公司亮点',
  'opName': 'and',
  'opProperty': '',
  'sonSize': 3,
  'relatedSize': '0',
  'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
  'source': 'text2sql'},
 {'dateText': '',
  'indexName': '所属概念',
  'indexProperties': [],
  'child_node': '所属概念',
  'ci': True,
  'source': 'text2sql',
  'type': 'index',
  'indexPropertiesMap': {},
  'reportType': 'null',
  'ciChunk': '所属概念',
  'createBy': 'preCache',
  'uiText': '所属概念',
  'valueType': '_所属概念',
  'domain': 'abs_股票领域',
  'sonSize': 0,
  'dateList': []},
 {'dateText': '',
  'indexName': '所属指数类',
  'indexProperties': [],
  'child_node': '所属指数类',
  'ci': True,
  'source': 'text2sql',
  'type': 'index',
  'indexPropertiesMap': {},
  'reportType': 'null',
  'ciChunk': '所属指数',
  'createBy': 'preCache',
  'uiText': '所属指数类',
  'valueType': '_所属指数类',
  'domain': 'abs_股票领域',
  'sonSize': 0,
  'dateList': []},
 {'dateText': '',
  'indexName': '公司亮点',
  'indexProperties': [],
  'child_node': '公司亮点',
  